In [1]:
# Preferably in a conda or pip virtual environment, or a Docker container even (so that
# this won't affect the globally installed Python packages):
# !pip install bqplot fastparquet ipywidgets pyarrow tqdm qgrid

In [1]:
import os

import ipywidgets as widgets
import pandas as pd
import qgrid
from bqplot import (
    Figure,
    Image,
    LinearScale,
    Scatter,
)


DATA_DIR = os.path.join("<YOUR_PATH_TO_THE_ROOT_OF_THE_DATA_DIR>", "twelve-respovision-CL-final", "Data")

PITCH_LEGNTH = 105.0
PITCH_WIDTH = 68.0
OFFSET = 3.0
OFFSET_LENGTH = OFFSET / PITCH_LEGNTH
OFFSET_WIDTH = OFFSET / PITCH_WIDTH
X = [-OFFSET_LENGTH, 1 + OFFSET_LENGTH]
Y_rev = [-OFFSET_WIDTH, 1 + OFFSET_WIDTH]

# Used to fix displaying quirks.
WIDTH = 506.7
HEIGHT = 346.7
FACTOR = 1.8

home_team = "Manchester City"
away_team = "Inter"

team_colours = {
    home_team: '#6cabdd',
    away_team: '#010E80',
}

ball_colour = 'orange'

def transform_x_coordinates(x):
    return x / PITCH_LEGNTH * 100

def transform_y_coordinates(x):
    return 100 - (x / PITCH_WIDTH * 100)


class RadarViewWidget(widgets.VBox):
    def __init__(self, pitch_img='pitch.png'):
        super().__init__()
        self.pitch_img = pitch_img
        self.image = self.__init_image()
        self.home_scatter = self.__init_scatter()
        self.away_scatter = self.__init_scatter()
        self.ball_scatter = self.__init_scatter(size=64, selected_opacity=1.0)
        
        self.fig = Figure(
            marks=[self.image, self.home_scatter, self.away_scatter, self.ball_scatter],
            padding_x=0, padding_y=0,
        )
        self.fig.layout = widgets.Layout(width=f'{WIDTH * FACTOR}px', height=f'{HEIGHT * FACTOR}px')
        self.output = widgets.Output()
        
        self.children = [self.fig, self.output]

    def __init_image(self):
        image_path = os.path.abspath(self.pitch_img)

        with open(image_path, 'rb') as f:
            raw_image = f.read()
        ipyimage = widgets.Image(value=raw_image, format='png')

        scales_image = {'x': LinearScale(), 'y': LinearScale(reverse=True)}
        axes_options = {'x': {'visible': True}, 'y': {'visible': True}}

        image = Image(image=ipyimage, scales=scales_image, axes_options=axes_options)

        image.x = X
        image.y = Y_rev        
        return image
        
    def __init_scatter(self, size=128, selected_opacity=0.6, unselected_opacity=1.0):
        scales={'x': LinearScale(min=X[0], max=X[1]), 'y': LinearScale(min=Y_rev[0], max=Y_rev[1], reverse=True)}
        axes_options = {'x': {'visible': False}, 'y': {'visible': False}}

        scatter = Scatter(
            scales= scales, 
            default_size=size,
            interactions={'click': 'select'},
            selected_style={'opacity': selected_opacity, 'stroke': 'Black'},
            unselected_style={'opacity': unselected_opacity},
            axes_options=axes_options,
        )
        scatter.enable_move = False
        return scatter

    def output_data(self, _, data):
        new_x = round(data['point']['x'], 2)
        new_y = round(data['point']['y'], 2)
        
        self.output.clear_output()
        with self.output:
            print(f'Changed player coordinates to ({new_x}, {new_y})')
    
    def set_data(self, frameset):
        self.home_scatter.x = frameset['home_x']
        self.home_scatter.y = frameset['home_y']
        self.home_scatter.names = frameset['home_names'],
        self.home_scatter.colors = frameset['home_color']
        
        self.away_scatter.x = frameset['away_x']
        self.away_scatter.y = frameset['away_y']
        self.away_scatter.names=frameset['away_names'],
        self.away_scatter.colors=frameset['away_color']
        
        self.ball_scatter.x = frameset['ball_x']
        self.ball_scatter.y = frameset['ball_y']
        self.ball_scatter.colors = frameset['ball_color']

def frames_from_respovision(df_tracks):
    assert df_tracks.period.unique().tolist() == [1, 2]

    # Map `frame` and `period` to consecutive integeres, starting from 0. This is needed because, unfortunately, the same `frame`
    # number may appear in both periods, otherwise.

    frames = {}
    frame_id = 0
    frame_map = {}
    for period in [1, 2]:
        for frame in df_tracks[df_tracks.period == period].frame.unique().tolist():
            filter_conds = (df_tracks.frame == frame) & (df_tracks.period == period)

            home_x = (transform_x_coordinates(df_tracks[filter_conds & (df_tracks.team_name == home_team)].x)/100).values
            home_y = (transform_y_coordinates(df_tracks[filter_conds & (df_tracks.team_name == home_team)].y)/100).values
            home_names = (df_tracks[filter_conds & (df_tracks.team_name == home_team)].player).tolist()
            home_numbers = (df_tracks[filter_conds & (df_tracks.team_name == home_team)].jersey_number).tolist()
            ball_x = (transform_x_coordinates(df_tracks[filter_conds & (df_tracks.player == "ball")].x)/100).values
            ball_y = (transform_y_coordinates(df_tracks[filter_conds & (df_tracks.player == "ball")].y)/100).values
            away_x = (transform_x_coordinates(df_tracks[filter_conds & (df_tracks.team_name == away_team)].x)/100).values
            away_y = (transform_y_coordinates(df_tracks[filter_conds & (df_tracks.team_name == away_team)].y)/100).values
            away_names = (df_tracks[filter_conds & (df_tracks.team_name == away_team)].player).tolist()
            away_numbers = (df_tracks[filter_conds & (df_tracks.team_name == away_team)].jersey_number).tolist()

            if len(home_names) + len(away_names) == 0: continue

            frame_map[(period, frame)] = frame_id
            frames[frame_id] = {
                'home_x': home_x,
                'home_y': home_y,
                'home_names': home_numbers,
                'home_color': [team_colours[home_team]],
                'ball_x': ball_x,
                'ball_y': ball_y,
                'ball_color': [ball_colour],
                'away_x': away_x,
                'away_y': away_y,
                'away_names': away_numbers,
                'away_color': [team_colours[away_team]],
            }
            frame_id += 1
    return frames, frame_map

In [2]:
match_id = 18768058
df_tracks = pd.read_parquet(os.path.join(DATA_DIR, f"{match_id}_tracks.parquet"))
frames, frame_map = frames_from_respovision(df_tracks)
df = df_tracks[['frame', 'period']].copy()
df.loc['frame_id'] = 0
for (frame, period), v in frame_map.items():
    df.loc[(df.period == frame) & (df.frame == period), 'frame_id'] = v

In [3]:
widget = RadarViewWidget()

initial_frame = 0
initial_frameset = frames[initial_frame]
widget.set_data(initial_frameset)

play = widgets.Play(
    value=0,
    step=1,
    max=len(df_tracks),
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider(max=max(frames.keys()))
widgets.jslink((play, 'value'), (slider, 'value'))

def change_data(change):
    widget.set_data(frames[change['new']])
    
slider.observe(change_data, names='value')
widgets.HBox([play, slider])

slider.value = initial_frame

qgrid.set_grid_option('maxVisibleRows', 10)
col_opts = { 
    'editable': False,
}

def on_row_selected(change):
    """callback for row selection: update selected points in scatter plot"""
    filtered_df = qgrid_widget.get_changed_df() 
    event = filtered_df.iloc[change.new]
    widget.set_data(frames[int(event['frame_id'].item())])
    slider.value = int(event['frame_id'].item())
   
        
qgrid_widget = qgrid.show_grid(df, show_toolbar=False, column_options=col_opts)
qgrid_widget.layout = widgets.Layout(width='920px')
   
qgrid_widget.observe(on_row_selected, names=['_selected_rows'])

display(widget)
display(widgets.HBox([play, slider]))
display(qgrid_widget)

RadarViewWidget(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(…

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…